In [1]:
!pip install -U scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import os
import sys
import utils
# import models
import torch
from tqdm import tqdm_notebook
from torch.utils.data import DataLoader
from torchvision import datasets
from gan import *
import torchvision.transforms as transforms
import torch.nn as nn

In [2]:
class arg_parser_subst():
    def __init__(self):
        self.n_epochs = 500 
        self.batch_size = 100
        self.batch_size_g = 100
        self.lr = 0.0002 
        self.b1 = 0.5 
        self.b2 = 0.999 
        self.n_cpu = 12 
        self.latent_dim = 100 
        self.img_size = 28
        self.img_shape = (1,28,28)
        self.channels = 1
        self.n_paths_G = 100 # number of generators
        self.classifier_para = 0.001
        self.classifier_para_g = 0.001
        self.logs_path='experiment'
        self.data_path='data'
        self.dataset='fmnist'
        self.architecture='cnn'
        self.logs_path='exp_ly_noisef'
        self.norm_d='layer_norm'
        self.init_noise=1.0
        self.noise_decay=0.01
        self.sample_interval=5
        self.no_conv_g = 2
        self.no_conv_d = 3
        self.lr_g = 0.0001
        self.lr_d = 0.0001
        self.lr_c = 0.0001
        
#--logs_path=experiment --n_paths_G=10 --data_path=data --dataset=fmnist\
#--architecture=cnn --logs_path=exp_ly_noisef --norm_d=layer_norm --init_noise=1.0 --noise_decay=0.01 --sample_interval=5

In [3]:
opt = arg_parser_subst()

In [ ]:
def main():
    # opt = utils.arg_parser_subst(sys.argv)
    ##opt = utils.parse_args()

    if opt.dataset == "fmnist":
        os.makedirs("data/fmnist", exist_ok=True)
        dataloader = torch.utils.data.DataLoader(
            datasets.FashionMNIST(
                "data/fmnist",
                train=True,
                download=True,
                transform=transforms.Compose(
                    [transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
                ),
            ),
            batch_size=opt.batch_size,
            shuffle=True,
        )
        if("cnn" in opt.architecture):
            import models_cnn_28 as models
        else:
            import models
        

    if opt.dataset == "mnist":
        os.makedirs("data/mnist", exist_ok=True)
        dataloader = torch.utils.data.DataLoader(
            datasets.MNIST(
                "data/mnist",
                train=True,
                download=True,
                transform=transforms.Compose(
                    [transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
                ),
            ),
            batch_size=opt.batch_size,
            shuffle=True,
        )
        if("cnn" in opt.architecture):
            import models_cnn_28 as models
        else:
            import models
        
        
    elif opt.dataset == "cifar10":
        os.makedirs("data/CIFAR10", exist_ok=True)
        dataloader = torch.utils.data.DataLoader(
            datasets.CIFAR10('data/CIFAR10', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize([.5],[.5])
                   ])
            ),
            batch_size=opt.batch_size, 
            shuffle=True)
        if("cnn" in opt.architecture):
            import models_cnn_32 as models
        else:
            import models

    generator = models.Generator(opt) #ok 
    discriminator = models.Discriminator(opt) # ok
    
    print("Generator: ", generator)
    print("Discriminator/classifier: ", discriminator)
    
    #optimizer G
    optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr_g, betas=(opt.b1, opt.b2))
    
    #optimizer D
    optimizer_D = torch.optim.Adam(list(discriminator.encoder_layers.parameters()) + list(discriminator.paths[0].parameters()), lr=opt.lr_d, betas=(opt.b1, opt.b2))
    # optimizer_D.add_param_group(discriminator.paths[0].parameters())
    
    #optimizer C
    optimizer_C = torch.optim.Adam(discriminator.paths[1].parameters(), lr=opt.lr_c, betas=(opt.b1, opt.b2))
    gan = Gan(generator, discriminator, optimizer_G, optimizer_D, optimizer_C, opt)

    gan.train(dataloader)

if __name__ == "__main__":
    main()

Generator:  Generator(
  (paths): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=100, out_features=12544, bias=True)
      (1): ReLU(inplace=True)
      (2): Reshape()
      (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (4): ReLU(inplace=True)
      (5): ConvTranspose2d(128, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (6): Tanh()
    )
    (1): Sequential(
      (0): Linear(in_features=100, out_features=12544, bias=True)
      (1): ReLU(inplace=True)
      (2): Reshape()
      (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (4): ReLU(inplace=True)
      (5): ConvTranspose2d(128, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (6): Tanh()
    )
    (2): Sequential(
      (0): Linear(in_features=100, out_features=12544, bias=True)
      (1): ReLU(inplace=True)
      (2): Reshape()
      (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 

In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))